In [1]:
import torch
from torchvision import transforms
import pandas as pd
import cv2
import datetime
import time
import face_recognition
import os

from src.model import MyModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando: {device}")

/home/miguel/Desktop/CV/.venv/lib/python3.12/site-packages/face_recognition_models/__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


Usando: cpu


## **CARGA DE DATOS**
Creamos el transform para modificar la imagen para la entrada del modelo, el modelo entrenado y creamos una lista con las emociones.

In [2]:
t = transforms.Compose([ # Transforms para convertir las imagenes a escala de grises a un tensor
    transforms.ToPILImage(),
    transforms.Resize((64,64)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

EMOTIONS = ["Angry", "Happy", "Neutral", "Sad"] # Lista de emociones

model = MyModel() # Cargamos nuestro modelo entrenado
model.load_state_dict(torch.load("models/model.pth", map_location=torch.device("cpu")))
model.to(device)

MyModel(
  (cnn): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.01)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.01)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LeakyReLU(negative_slope=0.01)
    (7): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.01)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.01)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=8192, out_features=256, bias=True)
   

## **INICIO**
Para el funcionamiento del programa, hay que añadir las caras que queremos identificar en la carpeta "./data/caras", siendo el nombre del archivo "nombre.*".

In [3]:
knownFaces = []
knownNames = []

def iniciarCaras(carpeta): # Cargamos las caras identificadas de la ruta especificada
    knownFaces.clear()
    knownNames.clear()
    for file in os.listdir(carpeta):
        path = os.path.join(carpeta, file)

        if os.path.isfile(path) and file.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')): # Procesamos solo imagenes
            name = os.path.splitext(file)[0].title()
            try:
                image = face_recognition.load_image_file(path)
                encoding = face_recognition.face_encodings(image)[0]

                if len(encoding) > 0:
                    knownFaces.append(encoding)
                    knownNames.append(name)
    
            except Exception as e:
                print(f"{file}. Error: {e}")

In [4]:
def guardarDato(nombre, emocion): # Función para almacenar un nuevo dato en el dataframe
    date = datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S")
    data = pd.DataFrame([{
        "Nombre": nombre, 
        "Fecha": date,
        "Emoción": emocion
    }])

    data.to_csv("resultados.csv", mode='a', header=False, index=False)

df = pd.DataFrame(columns=["Nombre", "Fecha", "Emoción"]) # Creamos un dataframe para almacenar el historial
df.to_csv("resultados.csv", index=False)

## **CARGAR USUARIOS IDENTIFICADOS**

In [6]:
carpeta = "data/identidades"
iniciarCaras(carpeta) # Iniciamos las caras identificadas

print(knownNames)

['Pedro']


## **RECONOCIMIENTO EN LOTE (FOTO)**
Para su funcionamiento hay que añadir en la carpeta "./data/input" la imagen de una cara en formato "nombrePersona.*"

In [8]:
files = [f for f in os.listdir("data/input") if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

for file in files:
    path = os.path.join("data/input", file) # Para cada imagen
       
    try:
        image = face_recognition.load_image_file(path)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Obtenemos la cara y lo pasamos a BGR
        
        faceLocations = face_recognition.face_locations(image, number_of_times_to_upsample=2) # Buscamos los pixeles de la cara
        faceEncodings = face_recognition.face_encodings(image, faceLocations) # Y su codificación 

        for (top, right, bottom, left), faceEncoding in zip(faceLocations, faceEncodings): # Para cada cara de la imagen
            matches = face_recognition.compare_faces(knownFaces, faceEncoding) # Comprobamos si existe ya en la carpeta identidades
            name = "Desconocido"
                
            if True in matches: # Si existe obtenemos el nombre de la persona
                firstMatch = matches.index(True)
                name = knownNames[firstMatch]
                
            face = image[top:bottom, left:right] # Recortamos la cara
            if face.size == 0: continue
                
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            x = t(face).to(device).unsqueeze(0) # Procesamos la cara para el modelo

            with torch.no_grad():
                y = model(x)
                
            prob = torch.nn.functional.softmax(y[0], dim=0) # Predecimos la salida aplicando softmax
            pred = torch.argmax(prob).item() # Obtenemos el índice de la emoción
            acc = prob[pred].item() # El accuracy de esa emoción
            emotion = EMOTIONS[pred] # La emoción predicha

            if name == "Desconocido": # Si no existe la persona la añadimos a nuestra carpeta de identidades
                name = os.path.splitext(file)[0].title()
                save_path = os.path.join("data/identidades", f"{name}.jpg")
                    
                cv2.imwrite(save_path, image)
                        
                knownFaces.append(faceEncoding)
                knownNames.append(name)
                        
            guardarDato(name, emotion) # Guardamos los datos en el csv
            print(f"Registrado con Nombre: {name}, Fecha: {datetime.datetime.now().strftime('%d-%m-%Y %H:%M:%S')}, Emoción: {emotion}") # Imprimimos por pantalla el resultado
                
        if len(faceLocations) > 0:
            os.remove(path) # Borramos de input los datos procesados
        else:
            print(f"No se detectaron caras en {file}. No se borrará el archivo.") # Si no hay caras avisamos

    except Exception as e:
        print(f"{file}. Error: {e}")

print("No hay más archivos")

Registrado con Nombre: Miguel, Fecha: 18-12-2025 19:55:19, Emoción: Neutral
No hay más archivos


## **RECONOCIMIENTO EN TIEMPO REAL (CÁMARA)**
Para su funcionamiento basta con ejecutar la celda inferior. Para acabar la ejecución pulsa "q". Se puede modificar cada cuanto tiempo se almacena un dato:

In [9]:
cap = cv2.VideoCapture(0)
TIEMPO = 5
lastTime = time.time()
inicio = time.time()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    currentTime = time.time()
    
    sframe = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25) # Reducimos el tamaño de la imagen para mayor rapidez (más eficiente)
    sframe = cv2.cvtColor(sframe, cv2.COLOR_BGR2RGB)

    faceLocations = face_recognition.face_locations(sframe) # Buscamos los pixeles de la cara
    faceEncodings = face_recognition.face_encodings(sframe, faceLocations) # Y su codificación 

    for (top, right, bottom, left), faceEncoding in zip(faceLocations, faceEncodings): # Para cada cara de la imagen
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4 # Reescalamos al tamaño original
        
        matches = face_recognition.compare_faces(knownFaces, faceEncoding) # Comprobamos si existe ya en la carpeta identidades
        name = "Desconocido"
        
        if True in matches: # Si existe obtenemos el nombre de la persona
            firstMatch = matches.index(True)
            name = knownNames[firstMatch]
        
        face = frame[top:bottom, left:right] # Recortamos la cara
        if face.size == 0: continue

        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        x = t(face).to(device).unsqueeze(0) # Procesamos la cara para el modelo

        with torch.no_grad():
            y = model(x)
            
        prob = torch.nn.functional.softmax(y[0], dim=0) # Predecimos la salida aplicando softmax
        pred = torch.argmax(prob).item() # Obtenemos el índice de la emoción
        acc = prob[pred].item() # El accuracy de esa emoción
        emotion = EMOTIONS[pred] # La emoción predicha

        if name != "Desconocido" and (currentTime - lastTime) >= TIEMPO: # Almacenamos los resultados cada TIEMPO segundos si tenemos a la persona identificada
            lastTime = currentTime
            guardarDato(name, emotion)

        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
        cv2.putText(frame, f"{name}: {emotion}, {acc*100:.1f}%", (left - 50, bottom + 50), cv2.FONT_HERSHEY_DUPLEX, 1.0, (0, 255, 0), 2)
    
    cv2.imshow("Camara", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()